<a href="https://colab.research.google.com/github/Praxis-QR/BDSN/blob/main/Parquet_Files_Superstore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

In [1]:
from datetime import datetime
import pytz
print('ॐ श्री सरस्वत्यै नमः',datetime.now(pytz.timezone('Asia/Calcutta')))
!python --version
!lsb_release -a

ॐ श्री सरस्वत्यै नमः 2023-09-11 20:39:07.394224+05:30
Python 3.10.12
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.2 LTS
Release:	22.04
Codename:	jammy


For a more detailed exploration of Pivot Tables please look at this [Notebook : Analysis of Retail Sales Data](https://github.com/Praxis-QR/PythonForManagers/blob/main/Retail_Sales_Data_Analysis.ipynb)

#SuperStore Data
Super Store Order Data https://drive.google.com/file/d/13MKOpo8iD-tMkeMLw-4IPFi5-xWhmqFZ/view?usp=sharing <br>
Original Data from [Tableau](https://community.tableau.com/docs/DOC-1236)

In [2]:
#!wget -O SS_Orders2.csv 'https://drive.google.com/uc?export=download&id=13MKOpo8iD-tMkeMLw-4IPFi5-xWhmqFZ'
#!gdown https://drive.google.com/uc?id=13MKOpo8iD-tMkeMLw-4IPFi5-xWhmqFZ
!wget -q -O Orders.csv 'https://raw.githubusercontent.com/Praxis-QR/RDWH/main/data/SS_Orders.csv'

In [3]:
!tail Orders.csv

9985,CA-2015-100251,17/05/2015,23/05/2015,Standard Class,DV-13465,Dianna Vittorini,Consumer,United States,Long Beach,New York,11561,East,OFF-LA-10003766,Office Supplies,Labels,Self-Adhesive Removable Labels,31.5,10,0,15.12
9986,CA-2015-100251,17/05/2015,23/05/2015,Standard Class,DV-13465,Dianna Vittorini,Consumer,United States,Long Beach,New York,11561,East,OFF-SU-10000898,Office Supplies,Supplies,"Acme Hot Forged Carbon Steel Scissors with Nickel-Plated Handles, 3 7/8"" Cut, 8""L",55.6,4,0,16.124
9987,CA-2016-125794,29/09/2016,03/10/2016,Standard Class,ML-17410,Maris LaWare,Consumer,United States,Los Angeles,California,90008,West,TEC-AC-10003399,Technology,Accessories,Memorex Mini Travel Drive 64 GB USB 2.0 Flash Drive,36.24,1,0,15.2208
9988,CA-2017-163629,17/11/2017,21/11/2017,Standard Class,RA-19885,Ruben Ausman,Corporate,United States,Athens,Georgia,30605,South,TEC-AC-10001539,Technology,Accessories,Logitech G430 Surround Sound Gaming Headset with Dolby 7.1 Technology,79.99,1,0,28.

#Read CSV into Pandas Dataframe

In [4]:
import numpy as np
import pandas as pd

In [5]:
df = pd.read_csv('Orders.csv', delimiter = ',')
#df.head(2)

In [7]:
df.info()
#df.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

##Comma, Double Quotes look good

In [8]:
df.iloc[9993]['Product Name']

'Acco 7-Outlet Masterpiece Power Center, Wihtout Fax/Phone Line Protection'

In [9]:
df.iloc[9992]['Product Name']

'It\'s Hot Message Books with Stickers, 2 3/4" x 5"'

##Date looks Bad, need to be fixed

In [10]:
# what is the first and last order date ?
# we have four years of data, but date format changes ...
df['Order Date'].describe
#df['ship_date'].describe

<bound method NDFrame.describe of 0       08/11/2016
1       08/11/2016
2       12/06/2016
3       11/10/2015
4       11/10/2015
           ...    
9989    21/01/2014
9990    26/02/2017
9991    26/02/2017
9992    26/02/2017
9993    04/05/2017
Name: Order Date, Length: 9994, dtype: object>

In [11]:
# Some of the dates are string like dd/mm/yyyy while others are dd-mm-yyyy
# this will cause problems in converting string to real dates
# So we move all dd-mm-yyyy to dd/mm/yyyy -- could have been the other way round also
#
df['Order Date'] = df['Order Date'].str.replace('-','/')
df['Ship Date'] = df['Ship Date'].str.replace('-','/')

In [12]:
# convert all dates to dd/mm/yyyy format
#
df['Ship Date'] = pd.to_datetime(df['Ship Date'],format='%d/%m/%Y')
df['Order Date'] = pd.to_datetime(df['Order Date'],format='%d/%m/%Y')
#df['order_date'].describe

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Row ID         9994 non-null   int64         
 1   Order ID       9994 non-null   object        
 2   Order Date     9994 non-null   datetime64[ns]
 3   Ship Date      9994 non-null   datetime64[ns]
 4   Ship Mode      9994 non-null   object        
 5   Customer ID    9994 non-null   object        
 6   Customer Name  9994 non-null   object        
 7   Segment        9994 non-null   object        
 8   Country        9994 non-null   object        
 9   City           9994 non-null   object        
 10  State          9994 non-null   object        
 11  Postal Code    9994 non-null   int64         
 12  Region         9994 non-null   object        
 13  Product ID     9994 non-null   object        
 14  Category       9994 non-null   object        
 15  Sub-Category   9994 n

In [14]:
df.to_csv('Orders2.csv', index=False)

#Pandas Dataframe to Parquet Format
Ref https://mungingdata.com/pyarrow/parquet-metadata-min-max-statistics/

In [15]:
df.to_parquet('Orders2.parquet')

In [16]:
# Note that parquet files are much smaller
!ls -al Orders2.*

-rw-r--r-- 1 root root 2312057 Sep 11 15:11 Orders2.csv
-rw-r--r-- 1 root root  470882 Sep 11 15:11 Orders2.parquet


In [17]:
import pyarrow.csv as pv
import pyarrow.parquet as pq

In [18]:
pf = pq.ParquetFile('Orders2.parquet')

In [19]:
pf.metadata

  created_by: parquet-cpp-arrow version 9.0.0
  num_columns: 21
  num_rows: 9994
  num_row_groups: 1
  format_version: 2.6
  serialized_size: 10777

In [20]:
pf.metadata.row_group(0)

  num_columns: 21
  num_rows: 9994
  total_byte_size: 650681

In [21]:
pf.metadata.row_group(0).column(1)

  file_offset: 112125
  file_path: 
  physical_type: BYTE_ARRAY
  num_values: 9994
  path_in_schema: Order ID
  is_stats_set: True
  statistics:
      has_min_max: True
      min: CA-2014-100006
      max: US-2017-169551
      null_count: 0
      distinct_count: 0
      num_values: 9994
      physical_type: BYTE_ARRAY
      logical_type: String
      converted_type (legacy): UTF8
  compression: SNAPPY
  encodings: ('RLE_DICTIONARY', 'PLAIN', 'RLE')
  has_dictionary_page: True
  dictionary_page_offset: 57750
  data_page_offset: 95899
  total_compressed_size: 54375
  total_uncompressed_size: 106440

In [22]:
pf.metadata.row_group(0).column(20)

  file_offset: 459992
  file_path: 
  physical_type: DOUBLE
  num_values: 9994
  path_in_schema: Profit
  is_stats_set: True
  statistics:
      has_min_max: True
      min: -6599.978
      max: 8399.976
      null_count: 0
      distinct_count: 0
      num_values: 9994
      physical_type: DOUBLE
      logical_type: None
      converted_type (legacy): NONE
  compression: SNAPPY
  encodings: ('RLE_DICTIONARY', 'PLAIN', 'RLE')
  has_dictionary_page: True
  dictionary_page_offset: 398914
  data_page_offset: 443641
  total_compressed_size: 61078
  total_uncompressed_size: 74893

#Parquet to Pandas Dataframe

In [23]:
dfFull = pd.read_parquet('Orders2.parquet')
dfFull.head(2)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820


In [34]:
selCols = ['Order ID','Order Date','Segment','Region','Category','Sales','Quantity','Profit']
dfSelect = pd.read_parquet('Orders2.parquet', columns=selCols)
dfSelect.head(2)

,Order ID,Order Date,Segment,Region,Category,Sales,Quantity,Profit
0,CA-2016-152156,2016-11-08,Consumer,South,Furniture,261.96,2,41.9136
1,CA-2016-152156,2016-11-08,Consumer,South,Furniture,731.94,3,219.5820


In [25]:
dfSelect.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Order ID    9994 non-null   object        
 1   Order Date  9994 non-null   datetime64[ns]
 2   Segment     9994 non-null   object        
 3   Region      9994 non-null   object        
 4   Category    9994 non-null   object        
 5   Sales       9994 non-null   float64       
 6   Quantity    9994 non-null   int64         
 7   Profit      9994 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 624.8+ KB


#PyArrow

In [26]:
import pyarrow.csv as pv
import pyarrow.parquet as pq

In [27]:
table = pv.read_csv('Orders2.csv')

In [28]:
table_df = table.to_pandas()
table_df.head(2)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820


In [29]:
# Date is still not clean
table_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

In [30]:
# convert all dates to dd/mm/yyyy format
#
#df['ship_date'] = pd.to_datetime(df['ship_date']).dt.strftime('%d/%m/%Y')
#df['order_date'] = pd.to_datetime(df['order_date']).dt.strftime('%d/%m/%Y')
#
table_df['Ship Date'] = pd.to_datetime(table_df['Ship Date'],format='%Y-%m-%d')
table_df['Order Date'] = pd.to_datetime(table_df['Order Date'],format='%Y-%m-%d')

In [31]:
table_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Row ID         9994 non-null   int64         
 1   Order ID       9994 non-null   object        
 2   Order Date     9994 non-null   datetime64[ns]
 3   Ship Date      9994 non-null   datetime64[ns]
 4   Ship Mode      9994 non-null   object        
 5   Customer ID    9994 non-null   object        
 6   Customer Name  9994 non-null   object        
 7   Segment        9994 non-null   object        
 8   Country        9994 non-null   object        
 9   City           9994 non-null   object        
 10  State          9994 non-null   object        
 11  Postal Code    9994 non-null   int64         
 12  Region         9994 non-null   object        
 13  Product ID     9994 non-null   object        
 14  Category       9994 non-null   object        
 15  Sub-Category   9994 n

In [35]:
#pq.write_table(table, 'Orders2_T.parquet')

In [33]:
#table3 = pv.read_csv('Orders2_T.parquet')

ArrowInvalid: ignored

In [36]:
# Note that parquet files are much smaller
!ls -al Orders2*.*

-rw-r--r-- 1 root root 2312057 Sep 11 15:11 Orders2.csv
-rw-r--r-- 1 root root  470882 Sep 11 15:11 Orders2.parquet
-rw-r--r-- 1 root root  456516 Sep 11 15:14 Orders2_T.parquet


#Spark
https://davidadrian.cc/python-hadoop/

In [37]:
!head Orders2.csv

Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",731.94,3,0.0,219.582
3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters by Universal,14.62,2,0.0,6.8714
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,Unite

In [54]:
df9 = pd.read_csv('Orders2.csv')
#df9.dtypes
df9.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

In [39]:
df9A = pd.read_parquet('Orders2.parquet')
#df9.dtypes
df9A.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Row ID         9994 non-null   int64         
 1   Order ID       9994 non-null   object        
 2   Order Date     9994 non-null   datetime64[ns]
 3   Ship Date      9994 non-null   datetime64[ns]
 4   Ship Mode      9994 non-null   object        
 5   Customer ID    9994 non-null   object        
 6   Customer Name  9994 non-null   object        
 7   Segment        9994 non-null   object        
 8   Country        9994 non-null   object        
 9   City           9994 non-null   object        
 10  State          9994 non-null   object        
 11  Postal Code    9994 non-null   int64         
 12  Region         9994 non-null   object        
 13  Product ID     9994 non-null   object        
 14  Category       9994 non-null   object        
 15  Sub-Category   9994 n

In [40]:
!pip3 -q install pyspark
from pyspark.sql import SparkSession
#spark9 = SparkSession.builder.master("local[*]").getOrCreate()
# note UI port switched from default 4040 to 4050 to avoid clash with ngrok
spark9 = SparkSession.builder.master("local[*]").config('spark.ui.port', '4050').getOrCreate()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
#pdfSS.to_csv('SS_Orders_Clean2.csv')

In [43]:
!tail Orders2.csv
#!tail Orders2.parquet --- unreadable

9985,CA-2015-100251,2015-05-17,2015-05-23,Standard Class,DV-13465,Dianna Vittorini,Consumer,United States,Long Beach,New York,11561,East,OFF-LA-10003766,Office Supplies,Labels,Self-Adhesive Removable Labels,31.5,10,0.0,15.12
9986,CA-2015-100251,2015-05-17,2015-05-23,Standard Class,DV-13465,Dianna Vittorini,Consumer,United States,Long Beach,New York,11561,East,OFF-SU-10000898,Office Supplies,Supplies,"Acme Hot Forged Carbon Steel Scissors with Nickel-Plated Handles, 3 7/8"" Cut, 8""L",55.6,4,0.0,16.124
9987,CA-2016-125794,2016-09-29,2016-10-03,Standard Class,ML-17410,Maris LaWare,Consumer,United States,Los Angeles,California,90008,West,TEC-AC-10003399,Technology,Accessories,Memorex Mini Travel Drive 64 GB USB 2.0 Flash Drive,36.24,1,0.0,15.2208
9988,CA-2017-163629,2017-11-17,2017-11-21,Standard Class,RA-19885,Ruben Ausman,Corporate,United States,Athens,Georgia,30605,South,TEC-AC-10001539,Technology,Accessories,Logitech G430 Surround Sound Gaming Headset with Dolby 7.1 Technology,79.99,1

In [55]:
#Inferred Correctly
sdf9A = spark9.read.csv('Orders2.csv', inferSchema=True,header = True, quote = '"')
sdf9A.dtypes

[('Row ID', 'int'),
 ('Order ID', 'string'),
 ('Order Date', 'date'),
 ('Ship Date', 'date'),
 ('Ship Mode', 'string'),
 ('Customer ID', 'string'),
 ('Customer Name', 'string'),
 ('Segment', 'string'),
 ('Country', 'string'),
 ('City', 'string'),
 ('State', 'string'),
 ('Postal Code', 'int'),
 ('Region', 'string'),
 ('Product ID', 'string'),
 ('Category', 'string'),
 ('Sub-Category', 'string'),
 ('Product Name', 'string'),
 ('Sales', 'string'),
 ('Quantity', 'string'),
 ('Discount', 'string'),
 ('Profit', 'double')]

In [56]:
#Inferred CORRECTLY
sdf9B = spark9.read.parquet('Orders2.parquet', inferSchema=True,header = True, quote = '"')
sdf9B.dtypes

[('Row ID', 'bigint'),
 ('Order ID', 'string'),
 ('Order Date', 'timestamp_ntz'),
 ('Ship Date', 'timestamp_ntz'),
 ('Ship Mode', 'string'),
 ('Customer ID', 'string'),
 ('Customer Name', 'string'),
 ('Segment', 'string'),
 ('Country', 'string'),
 ('City', 'string'),
 ('State', 'string'),
 ('Postal Code', 'bigint'),
 ('Region', 'string'),
 ('Product ID', 'string'),
 ('Category', 'string'),
 ('Sub-Category', 'string'),
 ('Product Name', 'string'),
 ('Sales', 'double'),
 ('Quantity', 'bigint'),
 ('Discount', 'double'),
 ('Profit', 'double')]

In [60]:
selCols = ['Order ID','Order Date','Segment','Region','Category','Sales','Quantity','Profit']
#df9C = spark9.read.parquet('Orders2.parquet',inferSchema=True,header = True, quote = '"').select('Order ID','Order Date','Segment','Region','Category','Sales','Quantity','Profit')
sdf9C = spark9.read.parquet('Orders2.parquet',inferSchema=True,header = True, quote = '"').select(selCols)
#df9.dtypes
sdf9C.dtypes

[('Order ID', 'string'),
 ('Order Date', 'timestamp_ntz'),
 ('Segment', 'string'),
 ('Region', 'string'),
 ('Category', 'string'),
 ('Sales', 'double'),
 ('Quantity', 'bigint'),
 ('Profit', 'double')]

In [ ]:
#we remove the CRLF character from the end of the row if it exists
!sed 's/\r//g' /content/SS_Orders_Clean.csv > SSdatafile.csv
# remove the first line containing headers from the file
#!sed -i -e "1d" datafile.csv
!head SSdatafile.csv

Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",731.94,3,0.0,219.582
3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters by Universal,14.62,2,0.0,6.8714
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,Unite

In [ ]:
# remove , and " from data
import pandas as pd
p_df = pd.read_csv('SSdatafile.csv')
#p_df = pd.read_csv('datafile.csv')
p_df2 = p_df.replace(',', ' ', regex=True)
p_df3 = p_df2.replace('"', '', regex=True)
p_df3.to_csv('SSdatafile2.csv',index=False,sep=',')
!head SSdatafile2.csv

Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,Hon Deluxe Fabric Upholstered Stacking Chairs  Rounded Back,731.94,3,0.0,219.582
3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters by Universal,14.62,2,0.0,6.8714
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United 

In [ ]:
data_file = 'SSdatafile2.csv'
sdfSS2 = spark9.read.csv(data_file,inferSchema=True, header=True)
sdfSS2.dtypes

[('Row ID', 'int'),
 ('Order ID', 'string'),
 ('Order Date', 'date'),
 ('Ship Date', 'date'),
 ('Ship Mode', 'string'),
 ('Customer ID', 'string'),
 ('Customer Name', 'string'),
 ('Segment', 'string'),
 ('Country', 'string'),
 ('City', 'string'),
 ('State', 'string'),
 ('Postal Code', 'int'),
 ('Region', 'string'),
 ('Product ID', 'string'),
 ('Category', 'string'),
 ('Sub-Category', 'string'),
 ('Product Name', 'string'),
 ('Sales', 'double'),
 ('Quantity', 'int'),
 ('Discount', 'double'),
 ('Profit', 'double')]

##Spark SQL

In [61]:
spark9.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [62]:
#air_quality_sdf.createOrReplaceTempView("air_quality_sdf")
sdf9C.createOrReplaceTempView("Orders_Temp")

In [63]:
spark9.sql("show tables").show()

+---------+-----------+-----------+
|namespace|  tableName|isTemporary|
+---------+-----------+-----------+
|         |orders_temp|       true|
+---------+-----------+-----------+



In [64]:
spark9.sql('select * from Orders_Temp limit 5').show()

+--------------+-------------------+---------+------+---------------+--------+--------+--------+
|      Order ID|         Order Date|  Segment|Region|       Category|   Sales|Quantity|  Profit|
+--------------+-------------------+---------+------+---------------+--------+--------+--------+
|CA-2016-152156|2016-11-08 00:00:00| Consumer| South|      Furniture|  261.96|       2| 41.9136|
|CA-2016-152156|2016-11-08 00:00:00| Consumer| South|      Furniture|  731.94|       3| 219.582|
|CA-2016-138688|2016-06-12 00:00:00|Corporate|  West|Office Supplies|   14.62|       2|  6.8714|
|US-2015-108966|2015-10-11 00:00:00| Consumer| South|      Furniture|957.5775|       5|-383.031|
|US-2015-108966|2015-10-11 00:00:00| Consumer| South|Office Supplies|  22.368|       2|  2.5164|
+--------------+-------------------+---------+------+---------------+--------+--------+--------+



In [66]:
sql_drop_table = """
drop table if exists praxisdb.orders_sparkhive
"""

sql_drop_database = """
drop database if exists praxisdb cascade
"""

sql_create_database = """
create database if not exists praxisdb
location '/content/praxisdb/'
"""

sql_create_table = """
create table if not exists praxisdb.orders_sparkhive
using parquet
as select *
from Orders_Temp
"""

#as select to_timestamp(date) as date_parsed, *

print("dropping database...")
result_drop_db = spark9.sql(sql_drop_database)

print("creating database...")
result_create_db = spark9.sql(sql_create_database)

print("dropping table...")
result_droptable = spark9.sql(sql_drop_table)

print("creating table...")
result_create_table = spark9.sql(sql_create_table)

dropping database...
creating database...
dropping table...
creating table...


In [67]:
spark9.sql("select * from praxisdb.orders_sparkhive").show(5)
#.select("date_parsed", "O_3").show(5)

+--------------+-------------------+---------+------+---------------+--------+--------+--------+
|      Order ID|         Order Date|  Segment|Region|       Category|   Sales|Quantity|  Profit|
+--------------+-------------------+---------+------+---------------+--------+--------+--------+
|CA-2016-152156|2016-11-08 00:00:00| Consumer| South|      Furniture|  261.96|       2| 41.9136|
|CA-2016-152156|2016-11-08 00:00:00| Consumer| South|      Furniture|  731.94|       3| 219.582|
|CA-2016-138688|2016-06-12 00:00:00|Corporate|  West|Office Supplies|   14.62|       2|  6.8714|
|US-2015-108966|2015-10-11 00:00:00| Consumer| South|      Furniture|957.5775|       5|-383.031|
|US-2015-108966|2015-10-11 00:00:00| Consumer| South|Office Supplies|  22.368|       2|  2.5164|
+--------------+-------------------+---------+------+---------------+--------+--------+--------+
only showing top 5 rows



In [71]:
spark9.sql("show tables").show()

+---------+-----------+-----------+
|namespace|  tableName|isTemporary|
+---------+-----------+-----------+
|         |orders_temp|       true|
+---------+-----------+-----------+



In [72]:
spark9.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
| praxisdb|
+---------+



In [73]:
spark9.sql("use praxisdb").show()

++
||
++
++



In [74]:
spark9.sql("show tables").show()

+---------+----------------+-----------+
|namespace|       tableName|isTemporary|
+---------+----------------+-----------+
| praxisdb|orders_sparkhive|      false|
|         |     orders_temp|       true|
+---------+----------------+-----------+



##Parquet to Pandas again

In [69]:
!ls /content/praxisdb/orders_sparkhive

part-00000-01e62cd7-c2ae-48ed-b52d-d439c943c488-c000.snappy.parquet  _SUCCESS


In [70]:
df7 = pd.read_parquet('/content/praxisdb/orders_sparkhive/part-00000-01e62cd7-c2ae-48ed-b52d-d439c943c488-c000.snappy.parquet')
df7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Order ID    9994 non-null   object        
 1   Order Date  9994 non-null   datetime64[ns]
 2   Segment     9994 non-null   object        
 3   Region      9994 non-null   object        
 4   Category    9994 non-null   object        
 5   Sales       9994 non-null   float64       
 6   Quantity    9994 non-null   int64         
 7   Profit      9994 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 624.8+ KB


#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself.
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)